In [54]:
from dotenv import load_dotenv
import os

load_dotenv()

TELE_TOKEN = os.getenv("TELE_TOKEN")
YT_BUILD_TOKEN = os.getenv("YT_BUILD_TOKEN")

### Take all video from last 7 days

In [51]:
def bot_msg_hafidz(text):
    import requests
    url = f"https://api.telegram.org/bot{TELE_TOKEN}/sendMessage"

    payload = {
        "text": '[LOCAL] ' + text,
        "disable_web_page_preview": False,
        "disable_notification": False,
        "reply_to_message_id": None,
        "chat_id": "1908911926"
    }
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json"
    }

    response = requests.request("POST", url, json=payload, headers=headers)

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd
ser = Service("C:\Program Files (x86)\chromedriver.exe")
op = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ser, options=op)

In [17]:
links = []
channel_name = []

In [18]:
# days_list = []
# for i in range(6,14):
#     days_list.append(f'{i} days ago')
# for i in range(2,4):
#     days_list.append(f'{i} weeks ago')
# days_list.append('1 month ago')
# days_list.append('2 months ago')
# days_list

In [19]:
days_list = ['21 hours ago','22 hours ago','23 hours ago','1 day ago']

#### Scrolling tiap channel

In [20]:
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from tqdm import tqdm

# channels = """CNNindonesiaOfficial
# kompastv
# tribunnews
# tvOneNews
# MetrotvnewsOfficial
# BeritaSatuChannel
# VIVAcoid
# CNBCIndonesia_ID
# OfficialiNews
# HarianKompasCetak
# mykompascom
# UCQA6NejSxQguRkD3L8eXHzA
# OfficialNETNews
# TribunMedanTV
# detikcom
# TribunJogjaOfficial
# PikiranRakyatDigital
# suaradotcom"""

# channels_decode = """CNNindonesiaOfficial
# kompastv
# tribunnews
# tvOneNews
# MetrotvnewsOfficial
# BeritaSatuChannel
# VIVAcoid
# CNBCIndonesia_ID
# OfficialiNews
# HarianKompasCetak
# kompas.com
# iNews_id
# OfficialNETNews
# TribunMedanTV
# detikcom
# TribunJogjaOfficial
# PikiranRakyatDigital
# suaradotcom"""

channels = """CNNindonesiaOfficial
kompastv"""

channels_decode = """CNNindonesiaOfficial
kompastv"""





channels = channels.split('\n')
channels_decode = channels_decode.split('\n')

for channel,decode in zip(tqdm(channels), channels_decode):
    start_time = time.time()
    if channel in ['UCQA6NejSxQguRkD3L8eXHzA','UCH_ElasO_yPy0WI3rAOUkQQ','UCpFqnctVbdqj1UketjDVz4Q']:
        driver.get(f'https://www.youtube.com/channel/{channel}/videos')
    elif channel in ['suaradotcom']:
        driver.get(f'https://www.youtube.com/user/{channel}/videos')
    else:
        driver.get(f'https://www.youtube.com/c/{channel}/videos')
    sc = 0
    while True:
        time.sleep(2)
        sc += 1080
        driver.execute_script("window.scrollTo(0, {})".format(sc))
        date_check = WebDriverWait(driver, 60).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,'#metadata-line span+span')))[-1].text
        time.sleep(1)
        print(date_check)
        
        if date_check.split()[1] == 'days' and int(date_check.split()[0]) > 2:
            time_passed = round(time.time() - start_time)
            print(f"{time_passed} seconds passed after crawling title in channel {decode}")
            break
        if date_check in days_list:
            time_passed = round(time.time() - start_time)
            print(f"{time_passed} seconds passed after crawling title in channel {decode}")
            break
            
    soup = bs(driver.page_source, 'lxml')
    links_html = soup.select('#video-title')

    for i in links_html:
        links.append(i.get('href'))
        channel_name.append(decode)


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

19 hours ago
19 hours ago
1 day ago
14 seconds passed after crawling title in channel CNNindonesiaOfficial



 50%|██████████████████████████████████████████                                          | 1/2 [00:14<00:14, 14.89s/it]

15 hours ago
15 hours ago
22 hours ago
11 seconds passed after crawling title in channel kompastv


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:26<00:00, 13.33s/it]


In [21]:
# cek linknya berapa banyak
print(f'Starting to get all data, total: {len(links)}')

Starting to get all data, total: 180


### Data Crawling

In [22]:
all_data = []

In [42]:
from __future__ import unicode_literals
from youtube_transcript_api import YouTubeTranscriptApi # API for caption
from googleapiclient.discovery import build # API for comment

import youtube_dl # API for downloading mp3
import re

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from pytube import YouTube # API for description

import urllib.request
import json
import urllib

driver_wait = 5


youtube = build('youtube','v3', developerKey=YT_BUILD_TOKEN)

for link, ch in zip(tqdm(links),channel_name):
    try:
        driver.get(f"https://www.youtube.com/{link}")
    except:
        print('driver problem')
    time.sleep(2)
    
    each_data = {}
    each_data.update({"video_link": f"https://www.youtube.com{link}"})
    try:
        params = {"format": "json", "url": link}
        url = "https://www.youtube.com/oembed"
        query_string = urllib.parse.urlencode(params)
        url = url + "?" + query_string
        with urllib.request.urlopen(url) as response:
            response_text = response.read()
            data = json.loads(response_text.decode())
            video_title = (data['title'])
        each_data.update({"video_title": video_title})
    except Exception as e:
        print(e)
        each_data.update({"video_title": '-'})
    try: 
        video_date = WebDriverWait(driver, driver_wait).until(EC.presence_of_element_located((By.CSS_SELECTOR,'#info-strings'))).text
        each_data.update({"video_date": video_date})
        print(video_date)
    except Exception as e:
        print(e)
        each_data.update({"video_date": '-'})
    try:
        view_count = WebDriverWait(driver, driver_wait).until(EC.presence_of_element_located((By.CSS_SELECTOR,'.view-count'))).text
        each_data.update({"view_count": view_count})
        print(view_count)
    except Exception as e:
        print(e)
        each_data.update({"view_count": '-'})
    try:
        like_count = WebDriverWait(driver, driver_wait).until(EC.presence_of_element_located((By.CSS_SELECTOR,'.yt-simple-endpoint>yt-formatted-string'))).text
        each_data.update({"like_count": like_count})
        print(like_count)
    except Exception as e:
        print(e)
        each_data.update({"like_count": '-'})
    try:
        video_desc = YouTube(link).description
        #video_desc = WebDriverWait(driver, driver_wait).until(EC.presence_of_element_located((By.XPATH,'/html/body/ytd-app/div/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[9]/div[2]/ytd-video-secondary-info-renderer/div/ytd-expander/div/div/yt-formatted-string'))).text
        each_data.update({"video_desc": video_desc})
    except Exception as e:
        print(e)
        each_data.update({"video_desc": '-'})
        

        
    try:
        transcriptList = YouTubeTranscriptApi.list_transcripts(re.findall(r'(?<=watch\?v\=).+', link)[0])
        dataCaption = []
        for transcript in transcriptList:
            for text in transcript.translate('id').fetch():
                dataCaption.append(text.get('text'))
        data_caption_str = ''
        for i in dataCaption:
            data_caption_str += i + " "
        each_data.update({"caption": data_caption_str})
    except Exception as e:
        print(e)
        each_data.update({"caption": '-'})
        
        try:
            def get_video_comments(**kwargs):
                comments = []
                results = youtube.commentThreads().list(**kwargs).execute()
                comment_dict = {
                    "authorDisplayName": '',
                    "authorChannelUrl": '',
                    "authorProfileImageUrl": '',
                    "textOriginal": '',
                    "likeCount": '',
                    "replies" : ''
                }
                while results:
                    for item in results['items']:
                        ds = item['snippet']['topLevelComment']['snippet']
                        for com_key in comment_dict.keys():
                            if com_key != 'replies':
                                comment_dict[com_key] = ds[com_key]
                            else:
                                try:
                                    comment_dict['replies'] = item['replies']['comments']
                                except:
                                    comment_dict['replies'] = []
                        comments.append(comment_dict.copy())
                    # Check if another page exists
                    if 'nextPageToken' in results:
                        kwargs['pageToken'] = results['nextPageToken']
                        results = youtube.commentThreads().list(**kwargs).execute()
                    else:
                        break
                return comments
            
            get_all_comment = get_video_comments(part='snippet,replies',videoId=re.findall(r'(?<=watch\?v\=).+', link)[0])
            each_data.update({"video_comment": get_all_comment})      
            
        except Exception as e:
            print(e)
            dict_comment_kosong = {'authorDisplayName': '',
            'authorChannelUrl': '',
            'authorProfileImageUrl': '',
            'textOriginal': '',
            'likeCount': 0,
            'replies': []}
            each_data.update({"video_comment": [dict_comment_kosong]})
    
    all_data.append(each_data)
    
    
print('done')

In [24]:
all_data_table = pd.DataFrame(all_data)
all_data_table

,video_link,video_title,video_date,view_count,like_count,video_desc,caption,video_comment,channel_name
0,https://www.youtube.com/watch?v=Ddy8ONr28Cg,Satu Keluarga di Samarinda Terjebak Kebakaran ...,"Apr 17, 2022",329 views,3,"Sebuah rumah di Jalan AW Syahrani, Samarinda, ...",Ayo kita akan format selanjutnya dan Hai bye b...,"[{'authorDisplayName': '7M FISHING', 'authorCh...",CNNindonesiaOfficial
1,https://www.youtube.com/watch?v=jQ6VK0hnnMU,Hapus Tato Gratis Saat Ramadahan,"Apr 17, 2022",351 views,8,Bagaimana Pelaksanaan Maupun Sejauh Mana Progr...,Hai ini amin amin Terima kasih banyak bapak wa...,"[{'authorDisplayName': 'Kevin Candra Dinata', ...",CNNindonesiaOfficial
2,https://www.youtube.com/watch?v=jKMPfU07MY0,Pasca-Serangan di Al Aqsa,"Apr 17, 2022",886 views,8,"Bagaimana Perkembangan, Maupun Kondisi Terkini...",Hai Hai terhenti tapi kita keras selalu kalau ...,"[{'authorDisplayName': 'Rдmzдй Кдdу́гov 🇷🇺', '...",CNNindonesiaOfficial
3,https://www.youtube.com/watch?v=tAiR5mrjxe8,Ramadan di Negeri Para Dewa,"Apr 17, 2022",653 views,11,Hadirnya Bulan Ramadan Selalu Dinantikan Oleh ...,Hai Terima kasih Anda masih bersama kami soal ...,[{'authorDisplayName': ' Hamas Official حماس '...,CNNindonesiaOfficial
4,https://www.youtube.com/watch?v=uZbBOmutRhY,Pengamat: Aspirasi Akar Rumput Partai Berbeda ...,"Apr 17, 2022","10,946 views",133,Kencang suara arus bawah partai persatuan pemb...,Hai kencang suara arus bawah Partai Hai selama...,"[{'authorDisplayName': 'Yogi Purnama', 'author...",CNNindonesiaOfficial
...,...,...,...,...,...,...,...,...,...
175,https://www.youtube.com/watch?v=aDc8WYLq3BA,Warga Pademangan Nyaris Saling Pukul Diduga Ak...,"Apr 16, 2022","4,510 views",35,"JAKARTA, KOMPAS.TV - Sebuah rumah di Pademanga...",di sebuah rumah di Pademangan Jakarta pompa el...,"[{'authorDisplayName': 'Vision TV, Anthony dek...",kompastv
176,https://www.youtube.com/watch?v=EV5YQQr5FPc,"Terlibat Kecelakaan Beruntun, Pengendara Motor...","Apr 16, 2022","13,088 views",52,"SIDOARJO, KOMPAS.TV - Enam orang terluka, dan ...",enam orang terluka dan empat di ditangani unit...,"[{'authorDisplayName': 'someone 089', 'authorC...",kompastv
177,https://www.youtube.com/watch?v=YI-90fhQgt4,"RSUD Dokter Soekardjo Terendam Banjir, Sejumla...","Apr 16, 2022","2,193 views",15,"TASIKMALAYA, KOMPAS.TV - Hujan deras yang meng...",hujan deras yang mengguyur wilayah Kota Tasikm...,"[{'authorDisplayName': 'tania amalia', 'author...",kompastv
178,https://www.youtube.com/watch?v=jeIWSbEBRCA,Kampung Cibogo Purwakarta Terendam Banjir Usai...,"Apr 16, 2022","1,987 views",15,"PURWAKARTA, KOMPAS.TV - Hujan deras yang turun...",hujan deras yang turun sepanjang Jumat terenda...,"[{'authorDisplayName': 'Nastain Tain', 'author...",kompastv


### Check for Caption

In [25]:
no_caption_df = all_data_table[all_data_table['caption'] == '-']
no_caption_df

,video_link,video_title,video_date,view_count,like_count,video_desc,caption,video_comment,channel_name
37,https://www.youtube.com/watch?v=yQIdRvQCT58,Ramainya Buka Puasa Bersama Saat Payung Masjid...,"Apr 16, 2022","1,606 views",10,"Masjid Nabawi di Madinah, Arab Saudi menjadi s...",-,[],CNNindonesiaOfficial
79,https://www.youtube.com/watch?v=Kl1OxZ78q0A,CNN Indonesia - Quiz,"Apr 15, 2022",486 views,6,Live streaming 24 jam: https://www.cnnindonesi...,-,"[{'authorDisplayName': 'elisabeth rumiyati', '...",CNNindonesiaOfficial
88,https://www.youtube.com/watch?v=KSLtrwdiHvM,CNN Indonesia - Memperingati Wafatnya Isa Almasih,"Apr 14, 2022","1,261 views",19,The Credits\n\nSr. D.O.P: Nugraha\nProducer: M...,-,"[{'authorDisplayName': 'Albertus Ronald I.B.',...",CNNindonesiaOfficial
105,https://www.youtube.com/watch?v=yL8jMx6uIeE,Rusia Klaim Hancurkan Pabrik Produksi Tank di ...,"Apr 16, 2022","24,940 views",234,"KYIV, KOMPAS.TV - Pasukan Rusia menghancurkan ...",-,"[{'authorDisplayName': 'Elisabet', 'authorChan...",kompastv
106,https://www.youtube.com/watch?v=nLayyEK7Joo,Update Corona 16 April 2022: Bertambah Sebanya...,"Apr 16, 2022","3,460 views",,"JAKARTA, KOMPAS.TV - Satuan Tugas Penanganan C...",-,"[{'authorDisplayName': 'Wong Koclok', 'authorC...",kompastv
117,https://www.youtube.com/watch?v=kGrjZPTs81g,Kata Kapolri soal Kasus Korban Begal Amaq Sint...,"Apr 16, 2022","111,424 views",797,"JAKARTA, KOMPAS.TV Kapolri Jenderal Listyo Si...",-,"[{'authorDisplayName': 'silvia dewi', 'authorC...",kompastv
132,https://www.youtube.com/watch?v=Jsm2v8PJolc,Ratusan Rakyat Yordania Gelar Demonstrasi Keca...,"Apr 16, 2022","10,262 views",170,"AMMAN, KOMPAS.TV - Ratusan rakyat Yordania tur...",-,"[{'authorDisplayName': 'Winda Panjaitan', 'aut...",kompastv
135,https://www.youtube.com/watch?v=rpCvEBPUasg,"Tuntaskan Misi Luar Angkasa, Astronot Perempua...","Apr 16, 2022","9,498 views",87,"JAKARTA, KOMPAS.TV - Tiga astronot China kemba...",-,"[{'authorDisplayName': 'NKRI}★', 'authorChanne...",kompastv
164,https://www.youtube.com/watch?v=J2_bU6RMw1I,Zelensky Minta Uni Eropa Embargo Minyak Rusia:...,"Apr 16, 2022","8,164 views",53,"KIEV, KOMPAS.TV Presiden Ukraina Volodymyr Ze...",-,"[{'authorDisplayName': 'Anto Tok', 'authorChan...",kompastv
172,https://www.youtube.com/watch?v=SJ9eangh61s,"Gambar Satelit Terbaru, Konvoi Rusia Terus Ber...","Apr 16, 2022","58,423 views",488,"JAKARTA, KOMPAS.TV Maxar Technologies merilis...",-,"[{'authorDisplayName': 'HR123', 'authorChannel...",kompastv


In [26]:
print(f'There is {len(all_data_table) - len(no_caption_df)}/{len(all_data_table)} data!')

There is 170/180 data!


##### Save the data just in case

In [27]:

from datetime import datetime

now = datetime.now() # current date and time
date_time = now.strftime("%m/%d/%Y").replace('/','-')
date_time

'04-17-2022'

In [28]:
no_stt = f'data_yt_harian_{date_time}_raw.json'
with open(no_stt,'w') as f:
    all_data = json.dump(all_data_table.to_dict('records'),f)

### For data with no caption download the mp3

In [29]:
no_caption_df = all_data_table[all_data_table['caption'] == '-']
no_caption_df

,video_link,video_title,video_date,view_count,like_count,video_desc,caption,video_comment,channel_name
37,https://www.youtube.com/watch?v=yQIdRvQCT58,Ramainya Buka Puasa Bersama Saat Payung Masjid...,"Apr 16, 2022","1,606 views",10,"Masjid Nabawi di Madinah, Arab Saudi menjadi s...",-,[],CNNindonesiaOfficial
79,https://www.youtube.com/watch?v=Kl1OxZ78q0A,CNN Indonesia - Quiz,"Apr 15, 2022",486 views,6,Live streaming 24 jam: https://www.cnnindonesi...,-,"[{'authorDisplayName': 'elisabeth rumiyati', '...",CNNindonesiaOfficial
88,https://www.youtube.com/watch?v=KSLtrwdiHvM,CNN Indonesia - Memperingati Wafatnya Isa Almasih,"Apr 14, 2022","1,261 views",19,The Credits\n\nSr. D.O.P: Nugraha\nProducer: M...,-,"[{'authorDisplayName': 'Albertus Ronald I.B.',...",CNNindonesiaOfficial
105,https://www.youtube.com/watch?v=yL8jMx6uIeE,Rusia Klaim Hancurkan Pabrik Produksi Tank di ...,"Apr 16, 2022","24,940 views",234,"KYIV, KOMPAS.TV - Pasukan Rusia menghancurkan ...",-,"[{'authorDisplayName': 'Elisabet', 'authorChan...",kompastv
106,https://www.youtube.com/watch?v=nLayyEK7Joo,Update Corona 16 April 2022: Bertambah Sebanya...,"Apr 16, 2022","3,460 views",,"JAKARTA, KOMPAS.TV - Satuan Tugas Penanganan C...",-,"[{'authorDisplayName': 'Wong Koclok', 'authorC...",kompastv
117,https://www.youtube.com/watch?v=kGrjZPTs81g,Kata Kapolri soal Kasus Korban Begal Amaq Sint...,"Apr 16, 2022","111,424 views",797,"JAKARTA, KOMPAS.TV Kapolri Jenderal Listyo Si...",-,"[{'authorDisplayName': 'silvia dewi', 'authorC...",kompastv
132,https://www.youtube.com/watch?v=Jsm2v8PJolc,Ratusan Rakyat Yordania Gelar Demonstrasi Keca...,"Apr 16, 2022","10,262 views",170,"AMMAN, KOMPAS.TV - Ratusan rakyat Yordania tur...",-,"[{'authorDisplayName': 'Winda Panjaitan', 'aut...",kompastv
135,https://www.youtube.com/watch?v=rpCvEBPUasg,"Tuntaskan Misi Luar Angkasa, Astronot Perempua...","Apr 16, 2022","9,498 views",87,"JAKARTA, KOMPAS.TV - Tiga astronot China kemba...",-,"[{'authorDisplayName': 'NKRI}★', 'authorChanne...",kompastv
164,https://www.youtube.com/watch?v=J2_bU6RMw1I,Zelensky Minta Uni Eropa Embargo Minyak Rusia:...,"Apr 16, 2022","8,164 views",53,"KIEV, KOMPAS.TV Presiden Ukraina Volodymyr Ze...",-,"[{'authorDisplayName': 'Anto Tok', 'authorChan...",kompastv
172,https://www.youtube.com/watch?v=SJ9eangh61s,"Gambar Satelit Terbaru, Konvoi Rusia Terus Ber...","Apr 16, 2022","58,423 views",488,"JAKARTA, KOMPAS.TV Maxar Technologies merilis...",-,"[{'authorDisplayName': 'HR123', 'authorChannel...",kompastv


In [45]:
import youtube_dl
from tqdm import tqdm


for each_link in tqdm(no_caption_df['video_link'].to_list()):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'outtmpl': 'data01/' + each_link.split('/www.youtube.com/')[1] + '.mp3'
    }
    try:
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            dictMeta = ydl.extract_info(each_link,download=False)
            if dictMeta['duration'] < 1800:
                ydl.download([each_link])
    except:
        pass

### Speech to Text

#### Get all mp3 files

In [31]:
import requests
from os import listdir
from os.path import isfile, join

mypath = 'C:/Users/hafid/main/projects/data_science/youtube_scraping/data01'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles = ['data01/'+i for i in onlyfiles]
onlyfiles = [i for i in onlyfiles if '.mp3' in i]
onlyfiles

['data01/watch#v=J2_bU6RMw1I.mp3',
 'data01/watch#v=Jsm2v8PJolc.mp3',
 'data01/watch#v=kGrjZPTs81g.mp3',
 'data01/watch#v=Kl1OxZ78q0A.mp3',
 'data01/watch#v=KSLtrwdiHvM.mp3',
 'data01/watch#v=nLayyEK7Joo.mp3',
 'data01/watch#v=rpCvEBPUasg.mp3',
 'data01/watch#v=SJ9eangh61s.mp3',
 'data01/watch#v=yL8jMx6uIeE.mp3',
 'data01/watch#v=yQIdRvQCT58.mp3']

In [43]:
import os
from tqdm import tqdm
import speech_to_text
stt = []
for each_file in tqdm(onlyfiles):
    try:
        stt_out = speech_to_text.parse_transcription(each_file)
#         print(stt_out)
        stt.append(stt_out)
    except Exception as e:
        print(e)

In [33]:
len(stt)

10

In [34]:
stt

[{'transcription': 'percil', 'file': 'watch#v=J2_bU6RMw1I.mp3'},
 {'transcription': 'aku belum bisa keluar', 'file': 'watch#v=Jsm2v8PJolc.mp3'},
 {'transcription': [], 'file': 'watch#v=kGrjZPTs81g.mp3'},
 {'transcription': [], 'file': 'watch#v=Kl1OxZ78q0A.mp3'},
 {'transcription': [], 'file': 'watch#v=KSLtrwdiHvM.mp3'},
 {'transcription': [], 'file': 'watch#v=nLayyEK7Joo.mp3'},
 {'transcription': [], 'file': 'watch#v=rpCvEBPUasg.mp3'},
 {'transcription': [], 'file': 'watch#v=SJ9eangh61s.mp3'},
 {'transcription': [], 'file': 'watch#v=yL8jMx6uIeE.mp3'},
 {'transcription': [], 'file': 'watch#v=yQIdRvQCT58.mp3'}]

#### append all mp3 data that already been converted to text, to the dataframe

In [35]:
df = all_data_table # creating new dataframe, for comparison later

In [36]:
import re
for i in stt:
    if i['transcription']:
        print('https://www.youtube.com/' + i['file'].split('.mp3')[0].replace('#','?'))
        locate_link = df['video_link'] == 'https://www.youtube.com/' + i['file'].split('.mp3')[0].replace('#','?')
        df.loc[locate_link,['caption']] = i['transcription']

https://www.youtube.com/watch?v=J2_bU6RMw1I
https://www.youtube.com/watch?v=Jsm2v8PJolc


In [37]:
df

,video_link,video_title,video_date,view_count,like_count,video_desc,caption,video_comment,channel_name
0,https://www.youtube.com/watch?v=Ddy8ONr28Cg,Satu Keluarga di Samarinda Terjebak Kebakaran ...,"Apr 17, 2022",329 views,3,"Sebuah rumah di Jalan AW Syahrani, Samarinda, ...",Ayo kita akan format selanjutnya dan Hai bye b...,"[{'authorDisplayName': '7M FISHING', 'authorCh...",CNNindonesiaOfficial
1,https://www.youtube.com/watch?v=jQ6VK0hnnMU,Hapus Tato Gratis Saat Ramadahan,"Apr 17, 2022",351 views,8,Bagaimana Pelaksanaan Maupun Sejauh Mana Progr...,Hai ini amin amin Terima kasih banyak bapak wa...,"[{'authorDisplayName': 'Kevin Candra Dinata', ...",CNNindonesiaOfficial
2,https://www.youtube.com/watch?v=jKMPfU07MY0,Pasca-Serangan di Al Aqsa,"Apr 17, 2022",886 views,8,"Bagaimana Perkembangan, Maupun Kondisi Terkini...",Hai Hai terhenti tapi kita keras selalu kalau ...,"[{'authorDisplayName': 'Rдmzдй Кдdу́гov 🇷🇺', '...",CNNindonesiaOfficial
3,https://www.youtube.com/watch?v=tAiR5mrjxe8,Ramadan di Negeri Para Dewa,"Apr 17, 2022",653 views,11,Hadirnya Bulan Ramadan Selalu Dinantikan Oleh ...,Hai Terima kasih Anda masih bersama kami soal ...,[{'authorDisplayName': ' Hamas Official حماس '...,CNNindonesiaOfficial
4,https://www.youtube.com/watch?v=uZbBOmutRhY,Pengamat: Aspirasi Akar Rumput Partai Berbeda ...,"Apr 17, 2022","10,946 views",133,Kencang suara arus bawah partai persatuan pemb...,Hai kencang suara arus bawah Partai Hai selama...,"[{'authorDisplayName': 'Yogi Purnama', 'author...",CNNindonesiaOfficial
...,...,...,...,...,...,...,...,...,...
175,https://www.youtube.com/watch?v=aDc8WYLq3BA,Warga Pademangan Nyaris Saling Pukul Diduga Ak...,"Apr 16, 2022","4,510 views",35,"JAKARTA, KOMPAS.TV - Sebuah rumah di Pademanga...",di sebuah rumah di Pademangan Jakarta pompa el...,"[{'authorDisplayName': 'Vision TV, Anthony dek...",kompastv
176,https://www.youtube.com/watch?v=EV5YQQr5FPc,"Terlibat Kecelakaan Beruntun, Pengendara Motor...","Apr 16, 2022","13,088 views",52,"SIDOARJO, KOMPAS.TV - Enam orang terluka, dan ...",enam orang terluka dan empat di ditangani unit...,"[{'authorDisplayName': 'someone 089', 'authorC...",kompastv
177,https://www.youtube.com/watch?v=YI-90fhQgt4,"RSUD Dokter Soekardjo Terendam Banjir, Sejumla...","Apr 16, 2022","2,193 views",15,"TASIKMALAYA, KOMPAS.TV - Hujan deras yang meng...",hujan deras yang mengguyur wilayah Kota Tasikm...,"[{'authorDisplayName': 'tania amalia', 'author...",kompastv
178,https://www.youtube.com/watch?v=jeIWSbEBRCA,Kampung Cibogo Purwakarta Terendam Banjir Usai...,"Apr 16, 2022","1,987 views",15,"PURWAKARTA, KOMPAS.TV - Hujan deras yang turun...",hujan deras yang turun sepanjang Jumat terenda...,"[{'authorDisplayName': 'Nastain Tain', 'author...",kompastv


In [38]:
df.drop(df[df['caption'] == '-' ].index, inplace=True)

In [39]:
print(f'There is {len(df)}/{len(links)} data after speech to text!')

There is 172/180 data after speech to text!


In [40]:
save = f'data_yt_harian_{date_time}.json'
with open(save,'w') as f:
    json.dump(df.to_dict('records'),f)

In [41]:
bot_msg_hafidz(f'Youtube crawl local done, link: C:/Users/hafid/main/projects/data_science/youtube_scraping/{save}')

In [44]:
dir = 'C:/Users/hafid/main/projects/data_science/youtube_scraping/data01'
for f in os.listdir(dir):
    try:
        os.remove(os.path.join(dir, f))
    except:
        pass

### Caption Sentiment Classification

In [ ]:
# Under Construction

In [ ]:
# import pandas as pd
# with open(save) as f:
#     df= json.load(f)
# pd.DataFrame(df)

In [ ]:
# caption_sentiment = []
# for i in df['caption']:
#     sentiment = requests.post("http://172.17.61.220:8601/sentiment-batch", json = {'texts':[i]})
#     caption_sentiment.append(sentiment.json()['results'][0]['sentiment'])                                                                                                                                                  

In [ ]:
# df['caption_sentiment'] = caption_sentiment

In [ ]:
# df

In [ ]:
# with open('data_yt_caption_lengkap_sentiment_24-02-2022.json','w') as f:
#     json.dump(df.to_dict('records'),f)